In [2]:
import pandas as pd
import re
import time
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By

## Create Driver

In [299]:
driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()))

[WDM] - Downloading: 19.0kB [00:00, 3.77MB/s]                                   


## Scrape characters

In [306]:
sourse_url = "https://iceandfire.fandom.com/wiki/Category:Characters"

letters_list = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N",
                "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]

characters = []

for letter in letters_list:
    
    # go through each letter
    page_url = sourse_url + "?from=" + letter
    driver.get(page_url)
    character_elems = driver.find_elements(by=By.CLASS_NAME, value = 'category-page__member-link')
    
    # collect all character names and urls
    for character in character_elems:
        character_url = character.get_attribute('href')
        character_name = character.text
        characters.append({'character_name': character_name, "url": character_url})
    

## Scrape characters' info

In [309]:
# collect info for all characters

character_list = []
books_list = ["A Game of Thrones", "A Clash of Kings", "A Storm of Swords", "A Feast for Crows", 
              "A Dance with Dragons", "The Winds of Winter"]

for character in characters:
    
    print(character['character_name'])
    
    # go to character page
    driver.get(character['url'])
    
    # extract alias info
    alias = driver.find_elements(By.XPATH,'//div[@class="pi-item pi-data pi-item-spacing pi-border-color" and @data-source="alias"]')
    try:
        alias = alias[0].text[6:].replace('\n', ', ')
    except:
        alias = ""
    
    # extract allegiance info
    allegiance = driver.find_elements(By.XPATH,'//div[@class="pi-item pi-data pi-item-spacing pi-border-color" and @data-source="allegiance"]')
    try:
        allegiance = allegiance[0].text[11:].replace('\n', ', ')
    except:
        allegiance = ""
    
    # extract culture info
    culture = driver.find_elements(By.XPATH, '//div[@class="pi-item pi-data pi-item-spacing pi-border-color" and @data-source="culture"]')
    try:
        culture = culture[0].text[8:].replace('\n', ', ')
    except:
        culture = ""
    
    # extract books info
    books = driver.find_elements(By.XPATH, '//div[@class="pi-item pi-data pi-item-spacing pi-border-color" and @data-source="books"]')
    try:
        books = books[0].text[6:].replace('\n', ', ')
    except:
        books = ""
    
    if books == "":
        titles = driver.find_elements(By.TAG_NAME, "h3")
        book_list = []
        for title in titles:
            for book in books_list:
                if book in title.text:
                    book_list.append(book)
        if bool(len(book_list)):
            books = ', '.join(book_list)
        else:
            books = ""
        
    # add info to character list
    character_list.append({'character_name':character['character_name'], 'alias': alias, "allegiance": allegiance, "culture": culture, "books": books})
    
    

Addam Marbrand
Addison Hill
Adrack Humble
Adrian Tarbeck
Aegon Blackfyre
Aegon Frey
Aegon Frey (son of Aenys)
Aegon I Targaryen
Aegon II Targaryen
Aegon III Targaryen
Aegon IV Targaryen
Aegon Targaryen (Son of Aenys I)
Aegon Targaryen (son of Aerys II)
Aegon Targaryen (Son of Baelon)
Aegon Targaryen (son of Gaemon)
Aegon Targaryen (son of Jaehaerys I)
Aegon Targaryen (son of Rhaegar)
Aegor Rivers
Aelyx Targaryen
Aemma Arryn
Aemon (wildling)
Aemon Blackfyre
Aemon Targaryen (Son of Jaehaerys I)
Aemon Targaryen (Son of Maekar I)
Aemond Targaryen
Aenar Targaryen
Aenys Frey
Aerea Targaryen
Aerion Targaryen
Aerion Targaryen (son of Daemion)
Aeron Greyjoy
Aeryn Targaryen
Aerys Targaryen (son of Aegon)
Aethan Velaryon
Aethelmure
Aggo
Agnes Blackwood
Aladale Wynch
Aladore Florent
Alan Beesbury
Alan o' the Oak
Alan Tarly
Alannys Harlaw
Alaric Stark
Alarra Massey
Alarra Stark
Alayaya
Alayne Royce
Albar Royce
Albett
Albin Massey
Alchemist
Alekyne Florent
Alerie Hightower
Alester Florent
Alester No

Hal (Hairy)
Halder
Qhorin
Hali
Halleck
Halleck Hoare
Hallis Hornwood
Hallis Mollen
Hallyne
Halys Hornwood
Hareth
Harlan Hoare
Harlon Botley
Harlon Greyjoy
Harlon the Hunter
Harma
Harmen Uller
Harmond Umber
Harmund I Hoare
Harmund II Hoare
Harmund III Hoare
Harmund Sharp
Harrag Hoare
Harrag Sharp
Harras Harlaw
Harras Hoare
Harren Botley
Harren Hoare
Harren the Red
Harrion Karstark
Harrold Hardyng
Harrold Langward
Harrold Rogers
Harrold Swyft
Harrold Westerling
Harron Harlaw
Harry Rivers
Harry Strickland
Harwin
Harwin Strong
Harwyn Hoare
Harys Haigh
Harys Horpe
Harys Swyft
Helman Tallhart
Hendry Bracken
Henly (Maester)
Herndon of the Horn
Herrock Kenning
Heward
Hibald
High Lickspittle
High Septon (Pater)
High Septon (Tyrion's)
Gerold Hightower
Hilmar Drumm
Category:Historical characters
Hoarfrost Umber
Hobb
Hobber Redwyne
Hobert Hightower
Hodor
Hoke
Holly
Hop-Robin
Horas Redwyne
Horgan Hoare
Horton Redfort
Horys Hill
Hosman Norcross
Hosteen Frey
Hostella Frey
Hoster Blackwood
Hoster Frey

Robert Arryn
Robert Ashford
Robert Blackwood
Robert Blackwood (209)
Robert Brax
Robert Brax (son of Flement)
Robert Frey (son of Rhaegar)
Robert I Baratheon
Robert Reyne
Robert Strong
Robett Glover
Robin Darklyn
Robin Greyjoy
Robin Massey
Robin Ryger
Robyn Ryswell
Roderick Blackwood
Roderick Dustin
Roderick Stark
Rodrik Cassel
Rodrik Greyjoy
Rodrik Harlaw
Rodrik Sparr
Rodrik Stark
Rodwell Stark
Roger Corne
Roger of Pennytree
Roger Reyne
Rognar II Greyiron
Rohanne Tarbeck
Rohanne Webber
Roland Crakehall
Roland Crakehall (Kingsguard)
Roland I Arryn
Roland Westerling
Rolder
Rollam Westerling
Rolland Darklyn
Rolland Longthorpe
Rolley of Sisterton
Rolly Duckfield
Rolph Spicer
Romny Weaver
Ronel Rivers
Ronnal Baratheon
Ronnel Arryn (king)
Roone
Roose Bolton
Rorge
Roryn Drumm
Rosamund Ball
Rosamund Lannister
Rose of Red Lake
Rosey
Rosey (daughter of Aegon IV)
Roslin Tully
Rowan Gold-Tree
Roy Connington
Royce Blackwood
Rufus Leek
Rupert Brax
Rus
Russell Merryweather
Ryam Florent
Rycherd Crane


## Turn list into a data frame

In [313]:
characters_df = pd.DataFrame(character_list)

In [314]:
characters_df.head()

,character_name,alias,allegiance,culture,books
0,Addam Marbrand,,"House Marbrand, House Lannister, City Watch (f...","Westeros, Andals",A Feast for Crows
1,Addison Hill,The Bastard of Cornfield,"House Swyft, House Targaryen, Kingsguard of Ae...","Westeros, Andals",
2,Adrack Humble,,"House Humble, House Greyjoy","Westeros, Ironborn",A Dance with Dragons
3,Adrian Tarbeck,,"House Tarbeck, House Lannister, Greens","Westeros, Andals",
4,Aegon Blackfyre,,"House Blackfyre, House Targaryen (formerly)","Valyrians, With influence from the Andals",


## Clean data frame

In [3]:
characters_df = pd.read_csv('characters.csv')

In [4]:
characters_df

,Unnamed: 0.1,Unnamed: 0,character_name,alias,allegiance,culture,books
0,0,0.0,Addam Marbrand,NaN,"House Marbrand, House Lannister, City Watch (f...","Westeros, Andals",A Feast for Crows
1,1,1.0,Addison Hill,The Bastard of Cornfield,"House Swyft, House Targaryen, Kingsguard of Ae...","Westeros, Andals",NaN
2,2,2.0,Adrack Humble,NaN,"House Humble, House Greyjoy","Westeros, Ironborn",A Dance with Dragons
3,3,3.0,Adrian Tarbeck,NaN,"House Tarbeck, House Lannister, Greens","Westeros, Andals",NaN
4,4,4.0,Aegon Blackfyre,NaN,"House Blackfyre, House Targaryen (formerly)","Valyrians, With influence from the Andals",NaN
...,...,...,...,...,...,...,...
1470,1470,NaN,Tormund,"Tormund Giantsbane, Tall-talker, Horn-blower",Mance Rayder,Free folk,"A Storm of Swords, A Feast for Crows, A Dance ..."
1471,1471,NaN,Jon Snow,"Lord Snow, The Bastard of Winterfell, Lord Crow","House Stark, The Night's Watch",Northmen,"A Game of Thrones, A Clash of Kings, A Storm o..."
1472,1472,NaN,Tywin Lannister,"The great lion of the rock, The old lion",House Lannister,Westermen,"A Game of Thrones, A Clash of Kings, A Storm o..."
1473,1473,NaN,Missandei,NaN,House Targaryen,Naathi,"A Storm of Swords, A Feast for Crows, A Dance ..."


In [5]:
characters_df["books"].unique()

array(['A Feast for Crows', nan, 'A Dance with Dragons',
       'A Game of Thrones, A Clash of Kings, A Storm of Swords, A Dance with Dragons',
       'The World of Ice & Fire (mentioned), Fire & Blood (mentioned), The Princess and the Queen (mentioned), The Hedge Knight (mentioned), The Sworn Sword (mentioned), The Mystery Knight (mentioned), A Game of Thrones (mentioned), A Clash of Kings (mentioned), A Storm of Swords (mentioned), A Feast for Crows (mentioned), A Dance with Dragons (mentioned)',
       'The World of Ice & Fire (mentioned)',
       'The World of Ice & Fire (mentioned), Fire & Blood (mentioned), The Rogue Prince (mentioned), The Princess and the Queen (mentioned)',
       'A Game of Thrones, A Feast for Crows',
       'The World of Ice & Fire (mentioned), Fire & Blood (mentioned)The Rogue Prince (mentioned), The Princess and the Queen (mentioned)',
       'The World of Ice & Fire (mentioned), The Hedge Knight (appears), The Sworn Sword (mentioned), The Mystery Knight 

In [6]:
characters_df["books"] = re.sub(r'\(|\)', '', characters_df["books"])

TypeError: expected string or bytes-like object